In [24]:
# Core data manipulation and analysis
import contextily as ctx  # For basemaps
from scipy import stats
from pathlib import Path
import zipfile
import os
from urllib.parse import urlencode
import overpy  # For OpenStreetMap data
import json
import requests
import plotly.graph_objects as go
import plotly.express as px
from folium import plugins
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import CRS
import pyproj
from shapely.geometry import Point, Polygon, box
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
from dask.diagnostics import ProgressBar
import dask_geopandas
import pystac_client
import planetary_computer as pc

import pickle


In [2]:
# Define data paths
DATA_DIR = Path("../datasets")
PARCELS_SHP = DATA_DIR / "Westchester_County_Parcels" / "Westchester_Parcels.shp"
PARCELS_CSV = DATA_DIR / "Westchester_County_Parcels_3053716610941103405.csv"

# Load the shapefile (preferred for spatial analysis)
parcels_gdf = gpd.read_file(PARCELS_SHP)
parcels_gdf = parcels_gdf.loc[parcels_gdf.MUNI_NAME == 'Bedford']


c:\code\bedford-ubid\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: ..\datasets\Westchester_County_Parcels\Westchester_Parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [25]:
# Convert to WGS84 for web mapping if needed
if parcels_gdf.crs != 'EPSG:4326':
    print(f"\nConverting from {parcels_gdf.crs} to WGS84...")
    parcels_wgs84 = parcels_gdf.to_crs('EPSG:4326')
    print("Conversion complete")
else:
    parcels_wgs84 = parcels_gdf.copy()

bounds = parcels_wgs84.total_bounds  # [minx, miny, maxx, maxy]

bounds


Converting from EPSG:3857 to WGS84...
Conversion complete


array([-73.74815136,  41.16565413, -73.59163084,  41.28549615])

### download ms buildings

In [28]:

# Connect to Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace
)


# Get signed asset
search = catalog.search(
    collections=["ms-buildings"], 
    bbox=bounds, 
    limit=10
    )


# Check what items are available
items = list(search.items())
print(f"Found {len(items)} items for your area")

for i, item in enumerate(items):
    print(f"Item {i}: {item.id}")
    print(f"  Bbox: {item.bbox}")
    print(f"  Properties: {item.properties}")

Found 2 items for your area
Item 0: United States_2022-07-06
  Bbox: [-160.221701, 18.921459, -66.950461, 71.360579]
  Properties: {'title': 'Building footprints', 'datetime': '2022-07-06T00:00:00Z', 'description': 'Parquet dataset with the building footprints', 'table:columns': [{'name': 'geometry', 'type': 'byte_array', 'description': 'Building footprint polygons'}], 'table:row_count': 129554779, 'msbuildings:region': 'United States'}
Item 1: United States_2022-06-14
  Bbox: [-160.221701, 18.921459, -66.950461, 71.360579]
  Properties: {'title': 'Building footprints', 'datetime': '2022-06-14T00:00:00Z', 'description': 'Parquet dataset with the building footprints', 'table:columns': [{'name': 'geometry', 'type': 'byte_array'}], 'table:row_count': 129557523, 'msbuildings:region': 'United States'}


In [ ]:

# # Connect to Planetary Computer STAC API
# catalog = pystac_client.Client.open(
#     "https://planetarycomputer.microsoft.com/api/stac/v1",
#     modifier=pc.sign_inplace
# )
# # Get signed asset
# search = catalog.search(collections=["ms-buildings"], bbox=bounds)
# item = next(search.items())
# signed_item = pc.sign(item)
# asset = signed_item.assets['data']

# buildings = dask_geopandas.read_parquet(
#     asset.href,
#     storage_options=asset.extra_fields["table:storage_options"]
# )
# with open('../datasets/buildings.pickle', 'wb') as f:
#     pickle.dump(buildings, f)

In [10]:

# # Load data directly
# buildings = dask_geopandas.read_parquet(
#     asset.href,
#     storage_options=asset.extra_fields["table:storage_options"]
# )

In [ ]:
with open('../datasets/buildings.pickle', 'wb') as f:
    pickle.dump(buildings, f)

Building data access functions defined successfully!
